## Question Embedding extraction through VQA without images

In this notebook we're going to use a Recursive Neural Network for generating Question embeddings based on the question format and it's answer.

The goal of this is to extract array representation for the questions where similar questions are close.

### Load the data

In [16]:
import findspark
findspark.init(findspark.find())

In [17]:
import tensorflow as tf
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

True

In [18]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from nltk.tokenize import word_tokenize
from collections import Counter
from keras.preprocessing.sequence import *
from keras.models import *
from keras.layers import *
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

sns.set(style="ticks")

spark = SparkSession \
    .builder \
    .appName("QuestionRephrasing-AutoEncoder") \
    .config("spark.executor.memory", "14G")\
    .config("spark.driver.memory", "14G")\
    .config("spark.driver.maxResultSize", "14G")\
    .getOrCreate()

spark.sparkContext.setCheckpointDir('data/checkpoints')
questions = spark.read.parquet("data/processed/union/*")
questions.printSchema()
questions.count()

root
 |-- question: string (nullable = true)
 |-- answer: string (nullable = true)
 |-- image_id: string (nullable = true)
 |-- tokenized_question: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- question_len: double (nullable = true)
 |-- question_word_len: double (nullable = true)
 |-- first_word: string (nullable = true)



Py4JJavaError: An error occurred while calling o160.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 30.0 failed 1 times, most recent failure: Lost task 0.0 in stage 30.0 (TID 1131, DESKTOP-L8H23GJ, executor driver): java.io.FileNotFoundException: C:\Users\Luca Andrei\AppData\Local\Temp\blockmgr-168de7ef-aa0d-47f1-a715-bc59e2e27ccf\39\temp_shuffle_0d6adf53-7a2b-418b-8206-8d7db9afd944 (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:105)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:118)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:245)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(By

### Question filtering based on answer frequencies

we're dealing with a dataset that has a wide variety of answers. So for this we'll consider only the answers that appear often in order to reduce the classes predicted.

In [ ]:
answers_counts = questions.select('answer')\
    .toPandas()['answer']\
    .value_counts()
answers_counts = pd.DataFrame(list(zip(answers_counts.keys(), answers_counts.values)))
most_common_answers = answers_counts[answers_counts[1] > 600][0]
f"We're going to target only {most_common_answers.count()} answers"

In [ ]:
most_common_answers_df = spark.createDataFrame([[x] for x in most_common_answers.tolist()], ["answer"])

In [ ]:
print(f"Original question size: {questions.count()}, after filtering {questions.join(most_common_answers_df, 'answer').count()}")
questions = questions.join(most_common_answers_df, 'answer')

In [ ]:
max_word_len = int(questions.agg({"question_word_len": "max"}).collect()[0]["max(question_word_len)"])

f"Mmaximum number of word in a question is {max_word_len}."

### Build the vocabulary

In [ ]:
# Tokens vocabulary and mappers
tokens = questions.select('tokenized_question')\
    .rdd\
    .flatMap(lambda x: x['tokenized_question'])\
    .collect()

word_mapping = {}
word_mapping_reversed = {}
word_counter = Counter(tokens)
for idx, value in enumerate(word_counter):
    word_mapping[value] = idx
    word_mapping_reversed[idx] = value
    
f"Word mapping example for 'is': {word_mapping['is']}."

### Extract the word representation of input using the mapping above

In [ ]:
extract_word_embeddings = F.udf(lambda tokenized_question: [word_mapping[word] + 1 for word in tokenized_question], ArrayType(IntegerType()))

questions = questions.withColumn('question_word_embeddings', extract_word_embeddings(F.col('tokenized_question')))
questions.head(1)

In [ ]:
word_embeddings = questions.select('question_word_embeddings')\
    .rdd\
    .map(lambda x: x['question_word_embeddings'])\
    .collect()
word_embeddings = pad_sequences(word_embeddings, maxlen=max_word_len, dtype='int32', padding='post', truncating='pre', value=0.0)
word_embeddings[:1]

### Transform the answers targets to one-hot representation

In [ ]:
answer_tokens = questions.select('answer')\
    .rdd\
    .map(lambda x: x['answer'])\
    .collect()

nr_answers = len(set(answer_tokens))

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(answer_tokens)
onehot_encoder = OneHotEncoder(sparse=True)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)

y = onehot_encoder.fit_transform(integer_encoded)
y[:1]

### Model for extracting question embeddings

**Input:** question embedded (each token represents a number according to the above mapping) as sequences and padded

**Output:** answers as one-hot

For extracting the question embeddings, we'll train the model and then we'll drop the last layer.

In [ ]:
from keras.layers import Dropout


model = Sequential()
model.add(Embedding(len(word_mapping) + 1, 150, input_length=max_word_len, mask_zero=True))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(250, activation='relu', dropout=0.1, recurrent_dropout=0.1), input_shape=(max_word_len, 100)))
model.add(Dropout(0.3))
model.add(Dense(100))
model.add(Dense(nr_answers, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

### Compute answers class weights

Since there are lots of answers the dataset is very imbalanced. Thus for optimizing the training we're providing class weights so the model can adjust based on how many times the answer appeared.

In [ ]:
from sklearn.utils import class_weight

classes = list(set(answer_tokens))

class_weights = class_weight.compute_class_weight('balanced', classes, answer_tokens)

In [ ]:
weights = list(zip(label_encoder.fit_transform(classes), class_weights))
weights.sort(key=lambda x: x[0])

### Model training and checkpointing

In [ ]:
filepath="model-checkpoints/answer-embeddings/answer-embeddings-model-{epoch:02d}-{val_accuracy:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, mode='max')
callbacks_list = [checkpoint]
model.fit(word_embeddings, [y],
                epochs=25,
                batch_size=1000,
                shuffle=True,
                callbacks=callbacks_list,
                validation_split=0.3)